In [4]:
# python
import os, sys, pickle
from itertools import combinations_with_replacement
from collections import OrderedDict

# stats
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from statsmodels.api import OLS

# plot
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

# neural networks
import torch, torch.utils.model_zoo # required to load nets
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

# analysis code
from lib.transforms import VisualPriorRepresentation
from lib.functions_second_analysis import *

DATASET_NAMES               = ('places1', 'places2', 'oasis')
SCALE_NAMES                 = ('scale2','scale4','scale8','scale16','scale32')
STUDY_NAMES                 = ("short presentation", "long presentation", "complexity order")
BEHAVIOUR_NAMES             = ('study1_places1_short.csv','study2_places1.csv','study3_places2.csv','study4_oasis.csv')

PATH_IMAGES                 = '../images and ratings/imageversions_256'
PATH_RATINGS                = '../images and ratings/ratings'
PATH_INTEGRATION_VALUES     = '../data csv/integration'
PATH_IB_CORRELATIONS        = '../data csv/ibcorr'
PATH_IB_CORRELATIONS_BLOCKED= '../data csv/ibcorr blocked'

PATH_RESULTS                = '../results'
PATH_PLOTS                  = '../plots'

#VisualPrior.viable_feature_tasks
MODEL_NAMES = ('autoencoding','depth_euclidean','jigsaw','reshading',
               'edge_occlusion','keypoints2d','room_layout', #'colorization' currently not working
               'curvature','edge_texture','keypoints3d','segment_unsup2d',
               'class_object','egomotion','nonfixated_pose','segment_unsup25d',
               'class_scene','fixated_pose','normal','segment_semantic',
               'denoising','inpainting','point_matching','vanishing_point')

In [ ]:
def calculate_rdm(data: pd.DataFrame, correlation_type : str = "pearson"):
    """Calculate RDM with pearson/spearman correlation for every combination of columns

    Parameters
    ----------
    data: pd.DataFrame
        Input with data to correlate in the columns

    correlation_type: str
        Which correlation to use. "pearson" (default) or "spearman".

        
    Returns
    -------
    pd.DataFrame
        representational dissimilarity matrix of inputs' columns
    
    """
    num_columns = data.shape[1]

    # create empty matrix to store RDM
    # index and column labels are in order of input columns
    rdm = pd.DataFrame(np.full((num_columns, num_columns), np.nan), columns=data.columns, index=data.columns)
    
    for col1, col2 in combinations_with_replacement(data.columns, 2):
        # there's one NaN in the autoencoding integration values, filter this here, don't know why that happens
        co11_col2 = data[[col1,col2]].dropna()
        
        # calculate correlation
        if correlation_type == "pearson":
            corr = pearsonr(co11_col2.values[:,0], co11_col2.values[:,1])[0]
        elif correlation_type == "spearman":
            corr = spearmanr(co11_col2.values[:,0], co11_col2.values[:,1])[0]

        # fill upper and lower triangular matrix
        rdm.loc[col1, col2] = corr
        rdm.loc[col2, col1] = corr
        rdm.loc[col1, col1] = 0.0

    return rdm

# integration-beauty correlation in Taskonomy

nets order by peak integration beauty correlation

## import blocked integration beauty correlation for taskonomy models

In [ ]:
# load results
data_list = []


for model_name in MODEL_NAMES:
    for study_name in STUDY_NAMES:
        for scale_name in SCALE_NAMES:

            data = pd.read_csv(os.path.join(DATA_PATH, model_name, study_name, scale_name, 'ib_correlations.csv'), header=None)
            data.insert(0, 'scale', scale_name)
            data.insert(0, 'study',study_name)
            data.insert(0, 'model', model_name)

            data_list.append(data)
            #selfsimilarity.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'selfsimilarity.csv'), index=False, header=False)           
            #l2norm.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'l2norm.csv'), index=False, header=False)

# add layer labels
data_list = [data.reset_index().rename(columns={"index":"layer", 0:"ibcorr"}) for data in data_list]

# combine into one DataFrame
dfc = pd.concat(data_list).reset_index(drop=True).set_index(['model','study','scale','layer'])
dfc

## import blocked integration beauty p-values for taskonomy models

In [ ]:
# load results
data_list = []


for model_name in MODEL_NAMES:
    for study_name in STUDY_NAMES:
        for scale_name in SCALE_NAMES:

            data = pd.read_csv(os.path.join(DATA_PATH, model_name, study_name, scale_name, 'ib_correlations_pvalues.csv'), header=None)
            data.insert(0, 'scale', scale_name)
            data.insert(0, 'study',study_name)
            data.insert(0, 'model', model_name)

            data_list.append(data)
            #selfsimilarity.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'selfsimilarity.csv'), index=False, header=False)           
            #l2norm.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'l2norm.csv'), index=False, header=False)

# add layer labels
data_list = [data.reset_index().rename(columns={"index":"layer", 0:"ibcorr pvalue"}) for data in data_list]

# combine into one DataFrame
dfp = pd.concat(data_list).reset_index(drop=True).set_index(['model','study','scale','layer'])
dfp

## plot

In [ ]:
dfc_plot = (dfc
           .loc[:,"short presentation","scale8",:]
           .reset_index()
           .pivot(index="model", columns="layer", values="ibcorr"))

dfp_plot = (dfp
           .loc[:,"short presentation","scale8",:]
           .reset_index()
           .pivot(index="model", columns="layer", values="ibcorr pvalue"))

In [ ]:
sortorder = dfc_plot.max(axis=1).argsort().values[-1::-1]
dfc_plot = dfc_plot.iloc[sortorder,:]
dfp_plot = dfp_plot.iloc[sortorder,:]

In [ ]:
modelnames = list(dfc_plot.index.get_level_values("model"))

In [ ]:
# FDR correction
significant = dfp_plot < (0.05 / 17)
significant = ~significant
significant

In [ ]:
# only plot significant clusters
sns.heatmap(dfc_plot, yticklabels=modelnames, mask=significant)

## correlation differences of Taskonomy models

absoloute difference in correlation in each layer, summed up, normalized with 2 (spearman correlation range) * num_layers 

In [ ]:
# 
dfc= (dfc
      .loc[:,"short presentation","scale8",:]
      .reset_index()
      .pivot(index="model", columns="layer", values="ibcorr"))

In [ ]:
dfc

In [ ]:
# dissimilarity: 17 * 2 - sum of abs diff

In [ ]:
rdm = pd.DataFrame(np.full((len(MODEL_NAMES), len(MODEL_NAMES)), np.nan), columns=MODEL_NAMES, index=MODEL_NAMES)
print(rdm.shape)

for model1, model2 in combinations_with_replacement(MODEL_NAMES, 2):
    rdm.loc[model1, model2] = np.abs(dfc.loc[model1] - dfc.loc[model2]).sum() / (17 * 2)
    rdm.loc[model2, model1] = np.abs(dfc.loc[model2] - dfc.loc[model1]).sum() / (17 * 2)

rdm

In [ ]:
sns.heatmap(rdm, xticklabels=rdm.columns, yticklabels=rdm.index)

# Predictor 1: semantic-2d-3d
[Finished predictor RDM ](#predictor-rdm-semantic-2d-3d)

##### TODO <br>
> How to handle blocking of layers (take best of each block OR average) ?


In [ ]:
NETS_SEMANTIC = ['class_object','class_scene','segment_semantic']

# from radek paper missing: colorization (not downloadable from taskonomy)
NETS_2D = ['autoencoding','denoising','edge_texture','inpainting','keypoints2d','segment_unsup2d']

# from radek paper missing: z-depth (missing from importing as well) and distance (but this is not a network after all)
NETS_3D = ['edge_occlusion','keypoints3d','segment_unsup25d','reshading','normal','curvature']

NETS_ALL = NETS_SEMANTIC + NETS_2D + NETS_3D

## load integration data and beauty ratings

In [ ]:
# load results
data_list = []

for model_name in NETS_ALL:
    for dataset_name in DATASET_NAMES:
        data = pd.read_csv(os.path.join(PATH_INTEGRATION_VALUES, model_name, dataset_name, 'scale8', 'correlations.csv'), header=None)
        data = data.reset_index().rename({'index':'img id'}, axis=1)

        #data.insert(0, 'scale', scale_name)
        data.insert(0, 'dataset',dataset_name)
        data.insert(0, 'model', model_name)

        if model_name in NETS_SEMANTIC:
            data.insert(0, 'class', 'semantic')
        elif model_name in NETS_2D:
            data.insert(0, 'class', '2d')
        elif model_name in NETS_3D:
            data.insert(0, 'class', '3d')

        data_list.append(data)
        #selfsimilarity.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'selfsimilarity.csv'), index=False, header=False)           
        #l2norm.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'l2norm.csv'), index=False, header=False)

# convert correlation to integration
df_int = - pd.concat(data_list).set_index(['model','dataset','class','img id'])
df_int

In [ ]:
beauty_ratings = {}
for ratings_name in BEHAVIOUR_NAMES:
    
    data = (
        pd.read_csv(os.path.join(PATH_BEHAVIOUR, ratings_name),header=None)
        .mean(axis=1)
        .to_frame()
        .rename({0:'beauty rating'}, axis=1)
        )
    data.index.name = 'img_id'

    # add name of study to index
    beauty_ratings[ratings_name] = pd.concat([data], names=['dataset'], keys=[ratings_name])
    

### visualize average integration of layers

In [ ]:
df_int_netavg = df_int.groupby('model').mean().transpose()
df_int_netavg.head()

In [ ]:
handles, labels = df_int_netavg.plot().get_legend_handles_labels()

# already order legend by classes
order = [labels.index(netname) for netname in NETS_ALL]
plt.legend([handles[idx] for idx in order], [labels[idx] for idx in order], loc='center right',bbox_to_anchor = (1.5, .5))

#### grouped by semantic-2d-3d

In [ ]:
colors = len(NETS_SEMANTIC) * ['green'] + len(NETS_2D) * ['purple'] + len(NETS_3D) * ['orange']

In [ ]:
for (netname, int_netavg), color in zip(df_int_netavg.iloc[:,order].items(), colors):
    if netname in NETS_SEMANTIC:
        alpha = .7
    else:
        alpha = .3
    plt.plot(int_netavg, label=netname, color=color, alpha=alpha)
    plt.legend(loc='center right',bbox_to_anchor = (1.5, .5))

## model RDM

In [ ]:
# create model RDM for semantiv-2D-3D nets integration
model_rdm = pd.DataFrame(
        np.full((len(NETS_ALL), len(NETS_ALL)), np.nan),
        columns=NETS_ALL, index=NETS_ALL)

for combi in combinations_with_replacement(NETS_ALL,2):
    if combi in combinations_with_replacement(NETS_SEMANTIC,2) or \
        combi in combinations_with_replacement(NETS_2D,2) or \
        combi in combinations_with_replacement(NETS_3D,2):
        model_rdm.loc[combi] = 1
        model_rdm.loc[tuple(reversed(combi))] = 1
    else:
        model_rdm.loc[combi] = 0
        model_rdm.loc[tuple(reversed(combi))] = 0

sns.heatmap(model_rdm, cmap='viridis')

## correlate RDM with model-RDM

### single layer

In [ ]:
layer_id = 48

# fitler relevant data
layer_df = pd.DataFrame(df.loc[NETS_ALL,"places1", "scale8"][layer_id]).reset_index()
# needed for pivot into wide format
layer_df["img_id"] = layer_df.groupby("model").cumcount()

# pivot
layer_df = layer_df.pivot(columns="model", index="img_id", values=layer_id)

# reorder columns according to semantic-2D-3D nets
layer_df = layer_df[NETS_ALL]

rdm = calculate_rdm(layer_df, correlation_type="spearman")

pearsonr(rdm.values.flatten(), model_rdm.values.flatten())

In [ ]:
pearsonr(rdm.values.flatten(), model_rdm.values.flatten())

In [ ]:
sns.heatmap(rdm, cmap='viridis')

In [ ]:
xdm = rdm[rdm > .142].fillna(0)
sns.heatmap(xdm, cmap='viridis')

In [ ]:
xdm = rdm[rdm < 0].fillna(0)
sns.heatmap(xdm, cmap='viridis')

### all layers

In [ ]:
model_correlations = []
model_pvalues = []
# iterate layers
for layer_name, layer_series in df.loc[:,"places1", "scale8"].items():

    # put data back into DataFrame
    layer_df = pd.DataFrame(layer_series).reset_index()

    # needed for pivot into wide format
    layer_df["img_id"] = layer_df.groupby("model").cumcount()

    # pivot
    layer_df = layer_df.pivot(columns="model", index="img_id", values=layer_name)

    # reorder columns according to semantic-2D-3D nets
    layer_df = layer_df[NETS_ALL]

    rdm = calculate_rdm(layer_df, correlation_type="spearman")

    model_correlations.append(pearsonr(rdm.values.flatten(), model_rdm.values.flatten())[0])
    model_pvalues.append(pearsonr(rdm.values.flatten(), model_rdm.values.flatten())[1])

In [ ]:
alpha = 0.05

sns.lineplot(data=model_correlations)
plt.suptitle("Similarity in what is integrated")
plt.title("Correlation of taskonomy RDM with model (semantic-2D-3D) RDM")
plt.xlabel("Layer")
plt.ylabel("pearson correlation")


for x, layer_pvalue in enumerate(model_pvalues):
    if layer_pvalue < alpha:
        plt.scatter(x, 0, color='cyan', s=100, marker='o')


## variance partitioning of model classes

### average model classes

average integration values for each image from each category of networks

In [ ]:
df_int_classes = df_int.groupby(['dataset','img id', 'class']).mean()
df_int_classes

### convert to rank data
since the ib-correlation is the spearman correlation

just do OLS variance partitioning for now an then talk to daniel about it.

#### integration

In [ ]:
df_int_classes.columns.name = 'layer'
df_int_classes

In [ ]:
df_int_classes_ranks = (
    df_int_classes
    .unstack('class')
    .groupby('dataset')
    .rank()
    .astype(int)
)

df_icr = df_int_classes_ranks

df_icr

#### beauty

In [ ]:
df_beauty_ratings_rank = (
    pd.concat(beauty_ratings.values())
    .groupby('dataset')
    .rank()
    .astype(int))

df_brr = df_beauty_ratings_rank
df_brr

### create linear model

In [ ]:
# single study & layer
dataset = 'places1'
layer_idx = 48

df_icr.loc[dataset,layer_idx]

In [ ]:
for layer_id, layer in df_icr.groupby(level='layer', axis=1):
    pass

In [ ]:
for dataset_id, layer_dataset in layer.groupby(level='dataset', axis=0):
    pass

In [ ]:
layer_dataset

In [ ]:
df_icr.columns.get_level_values('layer')

In [ ]:
df_icr.index.get_level_values('dataset').unique()

In [ ]:
pd.DataFrame(np.nan,
             index=df_icr.index.get_level_values('dataset').unique(),
             columns=df_icr.columns.get_level_values('layer'))

In [ ]:
# how to present all these R2 values ?

In [ ]:
df_brr

In [ ]:
Y = df_brr.loc['study3_places2.csv',:].values
Y

In [ ]:
X = h.loc[:,(slice(None),'2d')].values
X

In [ ]:
OLS(Y, X).fit().rsquared

In [ ]:
def do_variance_partitioning():
    # do variance partitioning for one layer
    # i.e. for all unique, shared and full combinations of the three predictors
    # return dataframe with all R2 values
     

## predictor RDM (semantic-2d-3d)

The [model rdm](#model-rdm) is used as the predictors representing the semantic-2d-3d categorization

# predictor 2: integration in best layer
ordering of images by integration in best predicting layer

"what is integrated", alternatively average of correlation between in each layer, howevery layers may not correspond to each other, therefore best predicting layer is more general <br> <br>

`Interpretation`: The differences in absolout values correspond to how similar the "integration mechanism" in both networks are.<br> If we assume that beauty perception depends on a specific stage of processing and not the whole processing stream, then the best predicting layer of a network can be interpreted as the point, where the network best mimics the aspects of the processing that determine beauty. <br> 

If the a similar The value in Is there a single or are there different ways of predicting beauty ?`

In [ ]:
New predictor: Image representations of network

## get best predicting layer in each model

## correlate integration values of best predicting layers

# predictor 3:  integration profile across layers

RDM of RDM's that correlate integration ratings of each different layers inside each network.

[Finished predictor RDM](#predictor-rdm-layer-layer-similarity-inside-networks)

?: "how strong".

##### TODO
> Is this essentially the same thing as absoloute correlation differences alone ?


In [ ]:
# copy code for each models layerXlayer RDM
# correlate correlations using daniels code

## load integration data

Same [integration data](#load-integration-data-and-beauty-ratings) as before.

In [ ]:
df_int = df_int.droplevel('class') # don't need that here
df_int

## layer X layer RDM for each network

In [ ]:
layer_layer_rdms_places1 = {}
layer_layer_rdms_places2 = {}
layer_layer_rdms_oasis = {}

for model_name, model_integration in df_int.groupby('model'):
    layer_layer_rdms_places1[model_name] = calculate_rdm(model_integration.loc[(slice(None),'places1'),:])
    layer_layer_rdms_places2[model_name] = calculate_rdm(model_integration.loc[(slice(None),'places2'),:])
    layer_layer_rdms_oasis[model_name] = calculate_rdm(model_integration.loc[(slice(None),'oasis'),:])

## correlate network RDMs

### put RDMs into columns

throw out zeros on diagonal to avoid skewing correlation (standard RSA procedure)

In [ ]:
RDMs_places1  = pd.DataFrame(columns=layer_layer_rdms_places1.keys())

In [ ]:
for network_name, rdm in layer_layer_rdms_places1.items():
    # mark diagonal values (all zeros)for removal
    np.fill_diagonal(rdm.values, np.nan)
    RDMs_places1.loc[:,network_name] = rdm.values.flatten()

# removed marked diagonal values
RDMs_places1 = RDMs_places1.dropna()

In [ ]:
print("Should be (2353, 15)")
RDMs_places1.shape

### predictor RDM (layer-layer similarity inside networks)

In [ ]:
calculate_rdm(RDMs_places1)

# predictor 4: spatial integration

"how"

"where" or alternatively "what",  which is the same because its spatial integration. Check for correlation between the what (represented by the integration ratings).

## integration is localized

### run integration searchlight analysis

In [ ]:
# visualize node score distribution

In [ ]:
# visualize within layer heatmaps

# exemplars



In [ ]:
# spatial correlation per image per net, correlate these netXnet
# test if integration scores are still correlating to beauty

LOOK AT SEPERATE NOTEBOOK

# predictors explaining correlation differences
do for each study and each scale, to check if there is some consistency in which factors always comes out on top

In [ ]:
# variance partitioning between different predictors